# Introduksjon

I denne delen av kurset bruker vi Python for å utveksle informasjon med SSBs API. Python er et meget nyttig språk for datahåndtering og automatisering av dataflyt. Særlig hvis man skal innom flere programmer med dataene kan python være en fordel. Jupyter notebooks er et veldig nyttig verktøy for å lære seg arbeidsflyten i en slik prosess. Gå gjerne igjennom *en kort introduksjon til Jupyter* før du starter her.

## Installere python

Python består av mange ulike forhåndsprogrammerte funksjoner som brukere kan sette sammen slik de ønsker for å dekke sine behov. Disse funksjonene er gjerne samlet i biblioteker som enten er med i grunnpakken til Python ([python.org](https://www.python.org/)) eller i andre installasjoner som for eksempel [miniconda](https://docs.conda.io/en/latest/miniconda.html) eller [Anaconda](https://www.anaconda.com/). 

Mange velger å bruke Anaconda siden det stort sett sikrer at alt virker når man oppdaterer programvaren. Jeg liker å bruke miniconda - det er en lettvektsversjon av anaconda som tar mindre plass, men krever litt mer av brukeren.

## Steg 1

Først må vi importere de modeulene vi må ha tilgang til. I dette tilfellet trenger vi biblioteket som håndterer kommunikasjonen mot APIet (`requests`) og vi trenger å lagre teksten fra spørringsfilen i rett format (`ast`). Den siste modulen (`pyjstat`) håndterer jstat formatet. Det er ikke nødvendig å bruke her, men lenger nede vises det hvordan vi kan laste fra en jstat fil rett inn i en dataframe for de som ønsker å behandle data videre i Python.

Gå til neste celle og trykk shift + enter for å kjøre cellen (og slik gjøre tilgjengelig de modulene vi skal bruke videre i denne arbeidsflyten).


In [ ]:
import requests # Bibliotek som håndterer spørringer
import ast # Leser en tekstfil med spørringen til riktig format (json)

from pyjstat import pyjstat # Håndterer jstat data

## Steg 2

Etter å ha importert de nødvendige modulene skal vi sette opp spørringen. Da trenger vi URL til SSB sitt API for den aktuelle tabellen. For enkelhets skyld så lagrer vi den som en variabel. Da trenger vi bare endre variablene senere for å kjøre programmet igjen med nye tabeller.

I dette kurset skal vi bare sende én spørring om gangen. Med noen grep kan dette settes opp for å gå igjennom en rekke spørringer for å vedlikeholde ett oppdatert fillager. Mer om dette på Hurdalssjøen.


In [ ]:
# URL med tabellnummer
url = r'https://data.ssb.no/api/v0/no/table/07459'

# Lenke til spørring - mer
query = r"spørringer/07459.txt"

For at `requests` modulen skal kunne sette opp sprørringen, må vi laste spørringen fra teksten inn i en variabel. I `with open(query, "r")` er det viktig at `query` er lenken til spørringen i forrige celle. Bokstaven `"r"` betyr at det kun er lov å lese teksten, ikke skrive over. Den lille kodesnutten under lagrer nå spørringen i variabelen `json_query` og det er denne vi skal sende til SSB med `requests` modulen.

In [ ]:
with open(query, "r") as file:
    json_query = ast.literal_eval(file.read())

Da skal alt være klart for å sende avgårde spørringen til SSB. Naviger til neste celle og kjør spørringen. Husk å vente til det står et tall i `In []` - det betyr at utveksling av data er ferdig.

In [ ]:
x = requests.post(url, json=json_query)

Nå vi har sendt spørringen kan vi sjekke status. Responsen består av mer enn bare data. Det er også informasjon om selve transaksjonen. En viktig ting er statuskoden.

Status koder 2xx betyr vanligvis at alt har gått fint. 4xx betyr at det er en feil på klientsiden (vi har gjort en feil). Det kan for eksempel være syntaksfeil i spørringen (et glemt komma for eksempel). 5xx betyr at det er en feil på serversiden (altså at det er et problem på leverandørsiden). For mer informasjon om statuskoder kan du lese [her](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)

Naviger til neste celle og kjør den for å sjekke status.

In [ ]:
x.status_code

Vi kan også se hva som står i respons body, altså se den teksten vi fikk i svar på spørringen. Denne teksten kan variere etter hvilket respons format vi har etterspurt i spørringen. Det er mulig å endre spørringen i venstre kolonne. Kjør den neste cellen. Ikke særlig nyttig - men vi kan få en ide om hva vi har fått tilbake. `\n\r`karakterene her er kodene for linjeskift i tekstfilen som vi har etterspurt. Det vil si at når vi skriver det til en fil, vil det skiftes linje hver gang det står `\r\n`.

In [ ]:
x.text

## Steg 3

Til slutt vil vi gjerne lagre filen i vårt fillager. Da bruker vi metoden `response.iter_content()` som går igjennom responsteksten uten å bruke for mye minne. Dette skrives så til en .csv fil (ettersom det er det vi har i spørringen).

I første omgang skriver vi det enkelt til en .csv fil.

In [ ]:
with open('data\07459.csv', 'wb') as fd:
    for chunk in x.iter_content(chunk_size=128):
        fd.write(chunk)

Hvis du er i nettversjonen av dette programmet vil du nå finne filen i mappen som heter *data* på venstre side. Hvis du er på egen maskin, så legger den seg i data mappen i den mappen du har kjørt skriptet fra. Hvis du ikke har en data mappe må du opprette en.

## Ekstramateriale

I de neste cellene skal vi skrive responsen til en pandas dataframe (pandas importeres sammen med pyjstat modulen). Her er det ikke like godt dokumentert, men det er mulig å kjøre igjennom og se litt på resultatene. 

Her er det viktig at spørringen ber om "json-stat2" respons format.

In [ ]:
# Les til json-stat2 format fra spørring
ds = pyjstat.Dataset.read(x.text)

In [ ]:
df = ds.write('dataframe')

In [ ]:
df.head()